In [77]:
import re
import requests
import pandas as pd

In [82]:
with open('Archers.htm', encoding='UTF-8') as archer_file, open('Fighters.htm', encoding='UTF-8') as fighter_file:
    archer_content = archer_file.read()
    fighter_content = fighter_file.read()

In [83]:
webpages = re.findall(r'/wiki/.*?(?=")', archer_content)
webpages += re.findall(r'/wiki/.*?(?=")', fighter_content)

In [89]:
thrall_collection = []
for i in webpages:
    page = 'https://conanexiles.fandom.com'+i
    response = requests.get(page)
    melee_mul = re.search(r'(?<=Melee Multiplier</th>\\n<th>)\d*\.\d*', str(response.content))
    acc_mul = re.search(r'(?<=Accuracy Multiplier</th>\\n<th>)\d*\.\d*', str(response.content))
    if acc_mul and melee_mul:
        thrall_collection.append(
            {'name':re.search('(?<=/wiki/).*',page).group(),
             'page':page,
             'melee':float(melee_mul.group()),
             'acc':float(acc_mul.group())}
        )

In [126]:
df = pd.DataFrame(thrall_collection)
df.set_index('name', inplace=True)
df['page'] = df['page'].apply(lambda x: f'<a href="{x}">{x}</a>')

# Best melee multiplier

In [127]:
df.sort_values(by='melee', ascending=False).style

,page,melee,acc
name,,,
Captain,https://conanexiles.fandom.com/wiki/Captain,3.900000,1.200000
Ortha_the_Flayer,https://conanexiles.fandom.com/wiki/Ortha_the_Flayer,2.340000,1.650000
Absak_Highstrung,https://conanexiles.fandom.com/wiki/Absak_Highstrung,2.250000,2.160000
Bessie,https://conanexiles.fandom.com/wiki/Bessie,2.250000,2.160000
Jotha_Spinebreaker,https://conanexiles.fandom.com/wiki/Jotha_Spinebreaker,2.250000,2.160000
J%27Onn_Farseek,https://conanexiles.fandom.com/wiki/J%27Onn_Farseek,2.250000,2.160000
Cimmerian_Berserker,https://conanexiles.fandom.com/wiki/Cimmerian_Berserker,2.240000,2.100000
Accursed_Berserker,https://conanexiles.fandom.com/wiki/Accursed_Berserker,2.240000,2.100000
Spinas_the_marauder,https://conanexiles.fandom.com/wiki/Spinas_the_marauder,2.210000,2.200000


# Best accuracy multiplier

In [128]:
df.sort_values(by='acc', ascending=False).style

,page,melee,acc
name,,,
Mandughai_Hundred-arrow,https://conanexiles.fandom.com/wiki/Mandughai_Hundred-arrow,1.600000,2.520000
Markos_the_Accursed,https://conanexiles.fandom.com/wiki/Markos_the_Accursed,1.600000,2.520000
"Markos,_Bane_of_Beasts","https://conanexiles.fandom.com/wiki/Markos,_Bane_of_Beasts",1.600000,2.520000
Relic_Hunter_Treasure_Seeker_(Archer),https://conanexiles.fandom.com/wiki/Relic_Hunter_Treasure_Seeker_(Archer),1.300000,2.520000
Senja,https://conanexiles.fandom.com/wiki/Senja,1.600000,2.520000
Tma_the_Slithering,https://conanexiles.fandom.com/wiki/Tma_the_Slithering,1.600000,2.520000
Anos_Preyfinder,https://conanexiles.fandom.com/wiki/Anos_Preyfinder,1.600000,2.520000
Vais_the_Wayfaerer,https://conanexiles.fandom.com/wiki/Vais_the_Wayfaerer,1.600000,2.520000
Hosth_the_Venomous,https://conanexiles.fandom.com/wiki/Hosth_the_Venomous,1.600000,2.520000


# Best of average multiplier

In [129]:
df['avg'] = (df['acc'] + df['melee']) / 2
df.sort_values(by='avg', ascending=False).style

,page,melee,acc,avg
name,,,,
Captain,https://conanexiles.fandom.com/wiki/Captain,3.900000,1.200000,2.550000
Absak_Highstrung,https://conanexiles.fandom.com/wiki/Absak_Highstrung,2.250000,2.160000,2.205000
Relic_Hunter_Treasure_Seeker_(Fighter),https://conanexiles.fandom.com/wiki/Relic_Hunter_Treasure_Seeker_(Fighter),2.210000,2.200000,2.205000
Jotha_Spinebreaker,https://conanexiles.fandom.com/wiki/Jotha_Spinebreaker,2.250000,2.160000,2.205000
J%27Onn_Farseek,https://conanexiles.fandom.com/wiki/J%27Onn_Farseek,2.250000,2.160000,2.205000
Daicas_the_sharp,https://conanexiles.fandom.com/wiki/Daicas_the_sharp,2.210000,2.200000,2.205000
Dalinsia_Snowhunter,https://conanexiles.fandom.com/wiki/Dalinsia_Snowhunter,2.210000,2.200000,2.205000
Erii_the_Ravager,https://conanexiles.fandom.com/wiki/Erii_the_Ravager,2.210000,2.200000,2.205000
Kisthis_Fleshtearer,https://conanexiles.fandom.com/wiki/Kisthis_Fleshtearer,2.210000,2.200000,2.205000
